In [ ]:
using InvertedIndices, SparseArrays

#### Función para calcular la distancia euclidiana entre un punto y varios puntos en $\mathbb{R}^p$, verificando la dimensión con la macro `@assert`

In [ ]:
function distanciaEuc(x::Array, y::Array)
    @assert size(x, 2) == size(y, 2) "Las dimensiones no coinciden"
    return .√(sum((x.-y).^2, dims = 2))
end

#### Función para generar puntos en un círculo alrededor del origen

In [ ]:
function puntosAzar(n::Int64; d = "unif", μ = 0.0, σ = 10.0)
    θ = 2π*rand(n)          # \'Angulos aleatorios
    if d == "unif"
        r = σ*rand(n) .- μ  # Radios aleatorios
    else
        r = μ .+ σ*randn(n)    
    end
    xy = hcat(r.*cos.(θ),r.*sin.(θ))
    return xy
end

#### Definición del tipo `Neurona`

In [ ]:
using SparseArrays
struct Neurona
    n::Int64                               # N\'umero de puntos en P
    A::SparseMatrixCSC{Bool}               # Matriz de adyacencia
    p::Matrix{Real}                        # Posiciones
    Neurona(n,A,p) = n < 1 ? error("Se necesita al menos un nodo") : new(n, A, p)
    Neurona(n,A,p) = size(A) != (n,n) ? error("Se requiere una matriz de dimension "*string(n)*" x "*string(n)) : new(n,A,p)
    Neurona(n,A,p) = size(p) != (n,2) ? error("Se requiere una matriz de dimension "*string(n)*"x 2") : new(n,A,p)
end

In [ ]:
function crearNeurona(n::Int64; b = 0.5, d₀ = "unif", μ₀ = 0, σ₀ = 10)
    p = puntosAzar(n, d = d₀, μ = μ₀, σ = σ₀)
    r = adyacencia(p, b)
    return r
end

In [ ]:
function crearNeurona(p::Matrix; b = 0.5)
    r = adyacencia(p, b)
    return r
end

In [ ]:
function adyacencia(p::Matrix, b::Float64)
    n = size(p,1)
    A = spzeros(n+1,n+1)
    d0 = distanciaEuc([0 0],p)
    dist = (1+b) .* d0
    masCercano = findmin(dist)
    puntoMasCercano = masCercano[2][1] 
    nodoEntrante = puntoMasCercano + 1 
    dist[puntoMasCercano,1] = Inf
    A[1, nodoEntrante] = 1
    padres = [1]
    hijos = [nodoEntrante]
    distCamino = d0[:,1]    
    for _ in 1:(n-1)
        d1 = distanciaEuc(reshape(p[puntoMasCercano,:],(1,2)), p)
        dist = hcat(dist, d1  .+  b .* ( d1 .+ distCamino[puntoMasCercano]))
        dist[hijos .- 1, end] .= Inf 
        masCercano = findmin(dist)
        puntoMasCercano = masCercano[2][1]
        nodoEntrante = puntoMasCercano + 1
        nodoPadre = masCercano[2][2] == 1 ? 1 :  hijos[masCercano[2][2]-1]
        A[nodoPadre, nodoEntrante] = 1   
        dist[puntoMasCercano, 1:end] .= Inf; 
        push!(hijos, nodoEntrante)
        push!(padres, nodoPadre)       
        if nodoPadre > 1
            d2 = distanciaEuc(reshape(p[nodoPadre-1,:],(1,2)), reshape(p[nodoEntrante-1,:],(1,2)))
            distCamino[puntoMasCercano] = distCamino[nodoPadre-1] + d2[1,1]
        end
    end
    return Neurona(n+1, A, vcat([0 0], p))
end

In [190]:
using CairoMakie
CairoMakie.activate!()
function dibujarNeurona(x::Neurona)
    fig, ax, p = scatter(x.p[:,1],x.p[:,2]; markersize=4, color = :black,
                        figure=(; resolution=(400, 400)), 
                        axis=(; aspect=DataAspect()))
    hidespines!(ax)
    hidedecorations!(ax)
    conecciones = findnz(x.A)
    enlaces = size( conecciones[1], 1)
    for i in 1:enlaces
        e = [ conecciones[1][i], conecciones[2][i] ]
        lines!(x.p[e,1],x.p[e,2]; color = :black)
    end
    fig
end

dibujarNeurona (generic function with 1 method)

### Simulación

In [195]:
using CSV, DataFrames
a = puntosAzar(200; μ = 10.0, σ = 20.0)
CSV.write("neuronaSim.csv", DataFrame(a,:auto))

"neuronaSim.csv"

In [ ]:
n0 = crearNeurona(200, μ₀ = -10, σ₀ = 20, b = 1.2);

In [ ]:
n1 = crearNeurona(a, b = 0.0);
dibujarNeurona(n1)

In [ ]:
n2 = crearNeurona(a, b = 0.2);
dibujarNeurona(n2)

In [ ]:
n3 = crearNeurona(a);
dibujarNeurona(n3)

In [ ]:
n4 = crearNeurona(a, b = 1.2);
dibujarNeurona(n4)